# HTML Table Generator
This code creates HTML tables and lists for San Francisco Planning Neighborhood Profiles. The code uses the master data table created by running Master Table Constuctor.ipynb and other input resources that can be found in this repository. The resulted html files are saved under the 'output' folder of this repository and can be accessed via this repository's hosted webpage. 

## Import packages

In [ ]:
import numpy as np
np.__version__
np.__path__
import sys
sys.version_info

In [ ]:
# base libraries
import requests, json, os
import pandas as pd
import numpy as np
import sodapy
from collections import defaultdict
import geopandas
from sodapy import Socrata
from IPython.display import HTML

#graph libraries
#import plotly
#import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# map libraries
#import folium
#import branca.colormap as cm
#a

## 1. ACS 

### Set the years

In [ ]:
year = 2020
year_past = 2010

### Open html_table_lookup.csv file

In [ ]:
table_info = pd.read_csv(r'./lookup_tables/html_table_lookup.csv')
table_info.head()

### Load the master data table

In [ ]:
# get the master table 
data_all_neighborhood = pd.read_csv(r'./output/Neighborhood_'+'master_table_by_geo_{}_{}.csv'.format(year, year_past))
data_all_neighborhood.head()

In [ ]:
# add a column containing '-' for null value 
data_all_neighborhood['null']= ['-']*len(data_all_neighborhood)
data_all_neighborhood.head()

### Create helper functions

#### Table Creator function
This function subset the master data table based on the lists of attributes IDs in the html_table_lookup.csv for each html table.

In [ ]:
# create a table creator function 
def table_constructor(neighborhood, table_info, table_id):
    print(table_id)
    data_sub = data_all_neighborhood[data_all_neighborhood["nhood_url"].isin([neighborhood, 'sf'])]
    attribute_sets = table_info[table_info['Table_ID']==table_id]['Attribute_sets'].iloc[0]
    past = table_info[table_info['Table_ID']==table_id]['Past'].iloc[0]
    
    attributes1 = table_info[table_info['Table_ID']==table_id]['Attributes1'].iloc[0].split(", ")
    print(attributes1)
    
    row_names = table_info[table_info['Table_ID']==table_id]['Row_names'].iloc[0].split(", ")
    column_names = table_info[table_info['Table_ID']==table_id]['Column_names'].iloc[0].split(", ")
    data_sub1 = data_sub[attributes1]
    data_sub1_tp = data_sub1.T.reset_index().iloc[:, 1:3]
    
    
    if attribute_sets == 2:
        
        if past == 0:
            attributes2 = table_info[table_info['Table_ID']==table_id]['Attributes2'].iloc[0].split(", ")
            print(attributes2)
            data_sub2 = data_sub[attributes2]
            data_sub2_tp = data_sub2.T.reset_index().iloc[:, 1:3]

            df = pd.concat([data_sub1_tp, data_sub2_tp], axis=1)
            df = df*100
            df.columns = ['a', 'b', 'c', 'd']
            df[''] = row_names 
            cols = df.columns.tolist()
            cols = [cols[4], cols[0], cols[2], cols[1], cols[3]]
            df = df[cols]
            df.columns = ['']+ column_names
        
        
        elif past == 1:
            attributes2 = [x+'_10' for x in attributes1]
            data_sub2 = data_sub[attributes2] 
            data_sub2_tp = data_sub2.T.reset_index().iloc[:, 1:3]
            
            df = pd.concat([data_sub2_tp, data_sub1_tp], axis=1)
            df = df*100
            df.columns = ['a', 'b', 'c', 'd']
            df[''] = row_names 
            cols = df.columns.tolist()
            print(cols)
            cols = [cols[4], cols[0], cols[2], cols[1], cols[3]]
            df = df[cols]
            df.columns = ['']+ column_names
    
    else:
        
        df = pd.concat([data_sub1_tp, data_sub1_tp], axis=1)
        total1 = df.iloc[0, 0]
        total2 = df.iloc[0, 1]
        df.iloc[:, 2] = df.iloc[:, 0]/total1*100
        df.iloc[:, 3] = df.iloc[:, 1]/total2*100
        df.columns = ['a', 'b', 'c', 'd']
        df[''] = row_names 
        cols = df.columns.tolist()
        cols = [cols[4], cols[0], cols[2], cols[1], cols[3]]
        df = df[cols]
        df.columns = ['']+ column_names
    
    #df.reindex(row_names)
    print(df)
    
    return(df)
    

#### Table Styler function

In [ ]:
# create a table styler function
# doc: https://pandas.pydata.org/docs/user_guide/style.html
# doc: https://datascientyst.com/render-pandas-dataframe-html-table-keeping-style/
def table_styler(df, table_info, table_id, neighborhood):
    
    df.columns=[['', neighborhood, neighborhood, 'SF', 'SF'], df.columns]
    
    columns = df.columns
    
    # set CSS styles
    row_color1 = {  # for row hover use <tr> instead of <td>
        'selector': 'tr:nth-child(even)',
        'props': 'background-color: #fef2de'
        }
    row_color2 = {
        'selector': 'tr:nth-child(odd)',
        'props': 'background-color: #ffffff'
    }
    td_th = {
        'selector': 'td, th',
        'props': 'text-align: right; padding: 10px; font-family:Arial, sans-serif; font-size: 14px'
        }
    header_neigh = {
        'selector': 'th:not(.index_name)',
        'props': 'background-color: #fef2de; text-align: center;'
        }
    header_sf = {
        'selector': 'th[colspan="2"]:nth-child(3)',
        'props': 'background-color: #fef2de; text-align: center; color:#ce6301'
    }
    index = {
        'selector':'td:nth-child(1)',
        'props': 'font-weight: bold'
    }
    column_color = {
        'selector':'td:nth-child(4), td:nth-child(5), th:nth-child(4), th:nth-child(5)',
        'props':'color:#ce6301'
        }
    
    attribute_sets = table_info[table_info['Table_ID']==table_id]['Attribute_sets'].iloc[0]
    
    if attribute_sets == 2: # all columns contain % numbers 
        
        # create a Style object with the CSS styles
        html = df.style.set_table_styles([row_color1, row_color2, td_th, header_neigh, header_sf, index, column_color]) \
                .format(na_rep='MISSING', thousands=",",
                       formatter = {columns[1]:'{:,.2f}%'.format,
                         columns[2]:'{:,.2f}%'.format,
                         columns[3]:'{:,.2f}%'.format,
                         columns[4]: '{:,.2f}%'.format})\
                .hide_index()
        
    else: # the first and second column contains absolute numbers. the third and fourth columns contain % numbers. 
        
        # create a Style object with the CSS styles
        html = df.style.set_table_styles([row_color1, row_color2, td_th, header_neigh, header_sf, index, column_color]) \
                .format(na_rep='MISSING', thousands=",",
                       formatter = {columns[1]:"{:.0f}",
                         columns[2]:'{:,.2f}%'.format,
                         columns[3]:"{:.0f}",
                         columns[4]: '{:,.2f}%'.format})\
                .hide_index()
    
    #print(html)
    return(html)
    

### Create/Style Tables 

In [ ]:
nhood_url_list = data_all_neighborhood['nhood_url'].tolist()[:-1]
table_id_list = table_info['Table_ID'].tolist()

In [ ]:
for neighborhood in nhood_url_list:
    for table_id in table_id_list:
        df = table_constructor(neighborhood, table_info, table_id)
        html = table_styler(df, table_info, table_id, neighborhood)
        
        # save the Style object as a html file 
        with open(r'./output/html_tables/'+str(year)+'_'+neighborhood+'_'+table_id+'.html', 'w') as f:
            f.write(html.render())
            f.close()

In [ ]:
html

## 2 Cultural Districts / Historic Statements 

In [ ]:
neighborhood_list = data_all_neighborhood['nhood_url'].tolist()

In [ ]:
# load the historic context statements resource file 
historic= pd.read_csv(r'./resources/Historic_Context_Statements.csv')
historic['nhood_url'] = historic['Neighborhood'].str.replace('/','-')
historic.head()

In [ ]:
# load the cultural districts resource file
cultural_districts= pd.read_csv(r'./resources/cultural_districts.csv')
cultural_districts['nhood_url'] = cultural_districts['Neighborhood'].str.replace('/','-')
cultural_districts.head()

In [ ]:
# create a function to make an element clickable 
def make_clickable(url, name):
    return f'<a href="{url}">{name}</a>'



In [ ]:
# Cultural Districts

#set CSS styles
row_color1 = {  # for row hover use <tr> instead of <td>
        'selector': 'tr',
        'props': 'background-color: #fef2de'
        }
td_th = {
        'selector': 'td, th',
        'props': 'text-align: left; padding: 10px; font-family:Arial, sans-serif; font-size: 14px'
        }

# generate all tables for each neighborhood 
for neighborhood in neighborhood_list:
    df = cultural_districts[cultural_districts['nhood_url']==neighborhood]
    print(df)
    print(len(df))
    if len(df) >0:
        df['link'] = df.apply(lambda x: make_clickable(x['CD_URL'], x['Cultural Districts']), axis=1)
        df2 = df[['link']]

        html = df2.style.set_table_styles([row_color1, td_th])\
                    .hide_index()\
                    .hide_columns()


        # save the Style object as a html file 
        with open(r'./output/html_tables/'+str(year)+'_'+neighborhood+'_'+'cultural_districts.html', 'w') as f:
            f.write(html.render())
            f.close()
    else: 
        with open(r'./output/html_tables/'+str(year)+'_'+neighborhood+'_'+'cultural_districts.html', 'w') as f:
            f.write('<div>Not applied</div>')
            f.close()
                            
        print('pass')

In [ ]:
# Historic Context Statements 

# set CSS styles 
row_color1 = {  # for row hover use <tr> instead of <td>
        'selector': 'tr',
        'props': 'background-color: #fef2de'
        }
td_th = {
        'selector': 'td, th',
        'props': 'text-align: left; padding: 10px; font-family:Arial, sans-serif; font-size: 14px; width:300px'
        }

# generate and save tables for each neighborhood 
for neighborhood in neighborhood_list:
    df = historic[historic['nhood_url']==neighborhood]
    print(df)
    print(len(df))
    if len(df) >0:
        df['link'] = df.apply(lambda x: make_clickable(x['HCS_URL'], x['Historic Context Statement']), axis=1)
        df2 = df[['link']]

        html = df2.style.set_table_styles([row_color1, td_th])\
                    .hide_index()\
                    .hide_columns()


        # save the Style object as a html file 
        with open(r'./output/html_tables/'+str(year)+'_'+neighborhood+'_'+'historic_statement.html', 'w') as f:
            f.write(html.render())
            f.close()
    else: 
        with open(r'./output/html_tables/'+str(year)+'_'+neighborhood+'_'+'historic_statement.html', 'w') as f:
            f.write('<div>Not applied</div>')
            f.close()
        print('pass')

In [ ]:
html

## 3 Neighborhood Group Notification List

In [ ]:
# load the neighborhood groups list resource file 
neigh_group= pd.read_csv(r'./resources/neighborhood_notification_list.csv')
neigh_group['nhood_url'] = neigh_group['Neighborhood'].str.replace('/','-')
neigh_group.head()

In [ ]:
neigh_group = neigh_group[['ORGANIZATION', 'EMAIL', 'nhood_url']]

# set CSS styles
row_color1 = {  # for row hover use <tr> instead of <td>
        'selector': 'tr:nth-child(even)',
        'props': 'background-color: #fef2de'
        }
row_color2 = {
    'selector': 'tr:nth-child(odd)',
    'props': 'background-color: #ffffff'
}
td = {
    'selector': 'td, th',
    'props': 'text-align: right; padding: 10px; font-family:Arial, sans-serif; font-size: 14px'
    }
header_neigh = {
    'selector': 'th:not(.index_name)',
    'props': 'background-color: #fef2de; text-align: center;'
    }
header_sf = {
    'selector': 'th[colspan="2"]:nth-child(3)',
    'props': 'background-color: #fef2de; text-align: center; color:#ce6301'
}
index = {
    'selector':'td:nth-child(1)',
    'props': 'font-weight: bold'
}
column_color = {
    'selector':'td:nth-child(4), td:nth-child(5), th:nth-child(4), th:nth-child(5)',
    'props':'color:#ce6301'
    }

# define a function that makes emailto elements
def make_clickable2(email):
    return f'<a href="mailto:{email}">Email</a>'

# generate and save tables for each neighborhood 
for neighborhood in neighborhood_list:
    df = neigh_group[neigh_group['nhood_url']==neighborhood]
    print(df)
    print(len(df))
    if len(df) >0:
        df['Type'] = ['-']*len(df)
        df['Topic'] = ['-']*len(df)
        df['Representing Population'] = ['-']*len(df)
        #df['Funding Agency'] = ['-']*len(df)
        df['Contact Info'] = df.apply(lambda x: make_clickable2(x['EMAIL']), axis=1)
        df.columns = ['Title','Email', 'Neighborhood', 'Type', 'Topic', 'Representing Population', 'Contact Info']
        df = df[['Title', 'Type', 'Topic', 'Representing Population', 'Contact Info']]

        html = df.style.set_table_styles([row_color1, row_color2, td, header_neigh, header_sf])\
                    .hide_index()


        # save the Style object as a html file 
        with open(r'./output/html_tables/2020_'+neighborhood+'_'+'neigh_groups.html', 'w') as f:
            f.write(html.render())
            f.close()
    else: 
        with open(r'./output/html_tables/'+str(year)+'_'+neighborhood+'_'+'neigh_groups.html', 'w') as f:
            f.write('<div>Not applied</div>')
            f.close()
        print('pass')

In [ ]:
html